<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Classification and KNN with NHL data

_Authors: Joseph Nelson (DC)_

---

Below you will practice KNN classification on a dataset of NHL statistics.

You will be predicting the `Rank` of a team from predictor variables of your choice.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
# web location:
web_csv = 'https://raw.githubusercontent.com/josephofiowa/GA-DSI/master/NHL_Data_GA.csv'
local_csv = './datasets/NHL_Data_GA.csv'

### 1. Load the NHL data

In [4]:
# A:
nhl = pd.read_csv(local_csv)

### 2. Perform any required data cleaning. Do some EDA.

In [5]:
# A:
nhl.head()

,Team,PTS,Rank,TOI,GF,GA,GF60,GA60,GF%,SF,...,FF%,CF,CA,CF60,CA60,CF%,Sh%,Sv%,PDO,PIM
0,Washington10,121,1,2001:52:00,115,73,3.45,2.19,61.2,1112,...,51.3,2138,1935,64.1,58.0,52.5,10.34,93.03,1034,1269
1,Vancouver11,117,1,2056:14:00,94,72,2.74,2.10,56.6,1143,...,53.1,2144,1870,62.6,54.6,53.4,8.22,93.16,1014,985
2,San Jose10,113,1,1929:54:00,90,68,2.80,2.11,57.0,1065,...,50.9,1985,1876,61.7,58.3,51.4,8.45,93.46,1019,1195
3,Chicago10,112,1,2020:23:00,104,83,3.09,2.46,55.6,1186,...,58.1,2093,1572,62.2,46.7,57.1,8.77,90.44,992,966
4,Vancouver12,111,1,2052:02:00,86,74,2.51,2.16,53.8,1078,...,51.0,2085,1880,61.0,55.0,52.6,7.98,93.36,1013,1049


In [12]:
nhl.shape

(90, 28)

In [16]:
nhl.isnull().sum()

Team                         0
Points                       0
Rank                         0
Time_On_Ice                  0
Goals_For                    0
Goals_Against                0
GF_Per_60min                 0
GA_Per_60min                 0
Goals_For%                   0
Shots_For                    0
Shots_Against                0
SF_Per_60min                 0
SA_Per_60min                 0
Shots_For%                   0
Fenwick_For                  0
Fenwick_Against              0
Fenwick_For_Per_60min        0
Fenwick_Against_Per_60min    0
Fenwick_For%                 0
Corsi_For                    0
Corsi_Against                0
Corsi_For_Per_60min          0
Corsi_Against_Per_60min      0
Corsi_For%                   0
Shooting%                    0
Save%                        0
Shooting_and_Save%           0
Penalty Minutes              0
dtype: int64

In [6]:
nhl.columns

Index([u'Team', u'PTS', u'Rank', u'TOI', u'GF', u'GA', u'GF60', u'GA60',
       u'GF%', u'SF', u'SA', u'SF60', u'SA60', u'SF%', u'FF', u'FA', u'FF60',
       u'FA60', u'FF%', u'CF', u'CA', u'CF60', u'CA60', u'CF%', u'Sh%', u'Sv%',
       u'PDO', u'PIM'],
      dtype='object')

In [9]:
new_columns = ['Team','Points','Rank','Time_On_Ice','Goals_For','Goals_Against','GF_Per_60min','GA_Per_60min',
               'Goals_For%','Shots_For','Shots_Against','SF_Per_60min','SA_Per_60min','Shots_For%',
               'Fenwick_For', 'Fenwick_Against','Fenwick_For_Per_60min','Fenwick_Against_Per_60min','Fenwick_For%',
               'Corsi_For','Corsi_Against', 'Corsi_For_Per_60min','Corsi_Against_Per_60min','Corsi_For%','Shooting%',
               'Save%','Shooting_and_Save%', 'Penalty Minutes']
nhl.columns = new_columns

In [13]:
nhl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 28 columns):
Team                         90 non-null object
Points                       90 non-null int64
Rank                         90 non-null int64
Time_On_Ice                  90 non-null object
Goals_For                    90 non-null int64
Goals_Against                90 non-null int64
GF_Per_60min                 90 non-null float64
GA_Per_60min                 90 non-null float64
Goals_For%                   90 non-null float64
Shots_For                    90 non-null int64
Shots_Against                90 non-null int64
SF_Per_60min                 90 non-null float64
SA_Per_60min                 90 non-null float64
Shots_For%                   90 non-null float64
Fenwick_For                  90 non-null int64
Fenwick_Against              90 non-null int64
Fenwick_For_Per_60min        90 non-null float64
Fenwick_Against_Per_60min    90 non-null float64
Fenwick_For%                 90 n

In [10]:
nhl.head(5)

,Team,Points,Rank,Time_On_Ice,Goals_For,Goals_Against,GF_Per_60min,GA_Per_60min,Goals_For%,Shots_For,...,Fenwick_For%,Corsi_For,Corsi_Against,Corsi_For_Per_60min,Corsi_Against_Per_60min,Corsi_For%,Shooting%,Save%,Shooting_and_Save%,Penalty Minutes
0,Washington10,121,1,2001:52:00,115,73,3.45,2.19,61.2,1112,...,51.3,2138,1935,64.1,58.0,52.5,10.34,93.03,1034,1269
1,Vancouver11,117,1,2056:14:00,94,72,2.74,2.10,56.6,1143,...,53.1,2144,1870,62.6,54.6,53.4,8.22,93.16,1014,985
2,San Jose10,113,1,1929:54:00,90,68,2.80,2.11,57.0,1065,...,50.9,1985,1876,61.7,58.3,51.4,8.45,93.46,1019,1195
3,Chicago10,112,1,2020:23:00,104,83,3.09,2.46,55.6,1186,...,58.1,2093,1572,62.2,46.7,57.1,8.77,90.44,992,966
4,Vancouver12,111,1,2052:02:00,86,74,2.51,2.16,53.8,1078,...,51.0,2085,1880,61.0,55.0,52.6,7.98,93.36,1013,1049


In [14]:
nhl.describe()

,Points,Rank,Goals_For,Goals_Against,GF_Per_60min,GA_Per_60min,Goals_For%,Shots_For,Shots_Against,SF_Per_60min,...,Fenwick_For%,Corsi_For,Corsi_Against,Corsi_For_Per_60min,Corsi_Against_Per_60min,Corsi_For%,Shooting%,Save%,Shooting_and_Save%,Penalty Minutes
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,...,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,91.977778,2.022222,83.288889,83.288889,2.442222,2.444000,49.981111,1068.333333,1068.333333,31.252222,...,49.966667,1973.466667,1973.466667,57.735556,57.798889,49.972222,7.814556,92.182556,999.988889,990.966667
std,12.524114,0.820767,10.376339,9.694484,0.325331,0.313522,4.644554,95.929047,75.514118,2.237637,...,2.797913,176.468299,154.148928,4.124476,4.291106,2.844313,0.866942,0.928621,12.292772,178.049321
min,62.000000,1.000000,57.000000,64.000000,1.700000,1.730000,38.000000,815.000000,868.000000,25.800000,...,43.100000,1565.000000,1572.000000,49.500000,46.700000,43.700000,5.900000,89.830000,978.000000,689.000000
25%,82.250000,1.000000,76.000000,75.500000,2.232500,2.202500,46.825000,1011.500000,1022.250000,29.550000,...,47.775000,1855.250000,1877.000000,54.275000,54.600000,47.925000,7.235000,91.555000,992.000000,881.250000
50%,92.500000,2.000000,84.000000,84.000000,2.400000,2.495000,49.700000,1072.000000,1072.000000,31.400000,...,50.050000,1981.500000,1961.000000,58.050000,58.350000,50.400000,7.730000,92.250000,1000.500000,960.000000
75%,102.000000,3.000000,90.000000,89.000000,2.600000,2.670000,53.625000,1143.000000,1125.750000,32.775000,...,51.775000,2112.750000,2077.250000,60.850000,60.400000,52.000000,8.270000,92.870000,1007.750000,1101.500000
max,121.000000,3.000000,115.000000,107.000000,3.450000,3.240000,61.200000,1311.000000,1245.000000,35.600000,...,58.100000,2341.000000,2332.000000,64.900000,67.500000,57.100000,10.340000,93.940000,1034.000000,1515.000000


### 3. Set up the `Rank` variable as your target. How many classes are there?

In [18]:
nhl.Rank.unique()
# There are 3 classes

array([1, 2, 3])

In [17]:
# A:
y = nhl.Rank
y.value_counts()

3    31
2    30
1    29
Name: Rank, dtype: int64

### 4. What is the baseline accuracy?

In [35]:
# A:
baseline = y.value_counts()/y.count()
baseline

3    0.344444
2    0.333333
1    0.322222
Name: Rank, dtype: float64

### 5. Choose 4 features to be your predictor variables and set up your design matrix.

In [28]:
# A:
features = ['Points', 'Goals_For', 'Goals_Against', 'Shooting%']
X = nhl[features]

In [29]:
X.head()

,Points,Goals_For,Goals_Against,Shooting%
0,121,115,73,10.34
1,117,94,72,8.22
2,113,90,68,8.45
3,112,104,83,8.77
4,111,86,74,7.98


In [30]:
print X.shape

y.shape

(90, 4)


(90,)

### 6. Fit a `KNeighborsClassifier` with 1 neighbor using the target and predictors.

In [31]:
# A:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

### 7. Evaluate the accuracy of your model.
- Is it better than baseline?
- Is it legitimate?

In [33]:
# A:
from sklearn import metrics

y_pred_class = knn.predict(X)

print metrics.accuracy_score(y, y_pred_class)

1.0


### 8. Create a 50-50 train-test-split of your target and predictors. Refit the KNN and assess the accuracy.

In [36]:
# A:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=99, test_size=0.5)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

y_pred_class = knn.predict(X_test)
print metrics.accuracy_score(y_test, y_pred_class)

0.866666666667


### 9. Evaluate the test accuracy of a KNN where K == number of rows in the training data.

In [40]:
# A:
x_train, x_test, y_train, y_test = train_test_split(X,y, train_size = .90)

knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
metrics.accuracy_score(y_test, y_pred)

0.88888888888888884

### 10. Fit the KNN at values of K from 1 to the number of rows in the training data.
- Store the test accuracy in a list.
- Plot the test accuracy vs. the number of neighbors.

In [12]:
# A:


### 11. Fit KNN across different values of K and plot the mean cross-validated accuracy with 5 folds.

In [13]:
# A:

### 12. Standardize the predictor matrix and cross-validate across the different K.
- Plot the standardized mean cross-validated accuracy against the unstandardized. Which is better?
- Why?

In [14]:
# A: